**Imports**

In [1]:
import os
import zipfile
import boto3
from botocore.client import Config

**Retrieve MinIO credentials, initialize client and list all buckets**

In [2]:
# minio_endpoint = os.getenv('AWS_S3_ENDPOINT')
# access_key = os.getenv('AWS_ACCESS_KEY_ID')
# secret_key = os.getenv('AWS_SECRET_ACCESS_KEY')
# bucket_name = "models"
# region = os.getenv('AWS_DEFAULT_REGION')

minio_endpoint = 'http://poc-minio-service:9000'
access_key = 'minio'
secret_key = 'minio123'
bucket_name = "models"
region = 'eu-west-1'

s3_client = boto3.client(
    's3',
    endpoint_url=minio_endpoint,
    aws_access_key_id=access_key,
    aws_secret_access_key=secret_key,
    config=Config(signature_version='s3v4')
)

objects = s3_client.list_objects_v2(Bucket=bucket_name)
if 'Contents' in objects:
    print("Objects found in bucket")
else:
    print("  No objects found.")

Objects found in bucket


**Get project, experiment and run name from environmental variables**

In [3]:
# project_name_param = os.getenv('project_name')
# experiment_name_param = os.getenv('experiment_name')
# run_name_param = os.getenv('run_name')

project_name_param = 'fracture'
experiment_name_param = 'fracture-classification'
run_name_param = 'run-3'

**Download the production model and the new model to score against the production model**

In [5]:
current_directory = os.getcwd()
download_folder = f"{project_name_param}/production/"
production_folder = "production"
production_path = os.path.join(current_directory, production_folder)

if not os.path.exists(production_path):
    os.makedirs(production_path)

try:
    response = s3_client.list_objects_v2(Bucket=bucket_name, Prefix=download_folder)

    if 'Contents' in response:
        for obj in response['Contents']:
            object_name = obj['Key']
            base_name = os.path.basename(object_name)
            download_path = os.path.join(production_path, base_name)

            s3_client.download_file(bucket_name, object_name, download_path)
            print(f"Object '{object_name}' downloaded successfully as '{download_path}'")
    else:
        print(f"No objects found in the folder '{download_folder}'")

except Exception as e:
    print(f"Error occurred: {e}")

Object 'fracture/production/Classif-Exp-fracture-classification-Run-run-2.onnx' downloaded successfully as '/opt/app-root/src/Beroepsproduct/Models/Scoring/production/Classif-Exp-fracture-classification-Run-run-2.onnx'


In [6]:
newmodel_folder = "newmodel"
newmodel_path = os.path.join(current_directory, newmodel_folder)

if not os.path.exists(newmodel_path):
    os.makedirs(newmodel_path)

model_to_score = f"{project_name_param}/{experiment_name_param}/Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx"
download_path_model_to_score = os.path.join(newmodel_path, f"Classif-Exp-{experiment_name_param}-Run-{run_name_param}.onnx")

try:
    s3_client.download_file(bucket_name, model_to_score, download_path_model_to_score)
    print(f"Object {model_to_score} downloaded successfully to '{download_path_model_to_score}'")
except Exception as e:
    print(f"Error occurred: {e}")

Object fracture/fracture-classification/Classif-Exp-fracture-classification-Run-run-3.onnx downloaded successfully to '/opt/app-root/src/Beroepsproduct/Models/Scoring/newmodel/Classif-Exp-fracture-classification-Run-run-3.onnx'


**Zip the models in their respective folders for the next pipeline node**

In [ ]:
folders_to_zip = ['newmodel', 'production']
zip_filename = 'models.zip'

zip_path = os.path.join(current_directory, zip_filename)

def add_directory_to_zip(zipf, folder, folder_base):
    for root, dirs, files in os.walk(folder):
        for file in files:
            file_path = os.path.join(root, file)
            zipf.write(file_path, os.path.relpath(file_path, folder_base))

with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for folder in folders_to_zip:
        folder_path = os.path.join(current_directory, folder)
        if os.path.exists(folder_path):
            add_directory_to_zip(zipf, folder_path, current_directory)
        else:
            print(f"Folder '{folder}' does not exist and will be skipped.")

print(f"Folders '{', '.join(folders_to_zip)}' have been zipped into '{zip_path}'.")